In [1]:
import pandas as pd
from time import time
import xarray as xr
import numpy as np
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from scipy import stats
import matplotlib.patches as mpatches

In [2]:
! aws s3 ls dev-et-data/in/NorthAmerica/

                           PRE ETo/
                           PRE NDVI/
                           PRE PPT/
                           PRE Soil/
                           PRE Temp/
2021-03-11 21:05:58  364931586 global_water_mask_inland.tif
2021-03-12 20:28:22   78133884 water_NA.tif
2021-03-15 16:49:18  269133066 water_global.tif


In [3]:
def _get_year_month(product, tif):
    fn = tif.split('/')[-1]
    fn = fn.replace(product,'')
    fn = fn.replace('.tif','')
    fn = fn.replace('_','')
    print(fn)
    return fn

In [4]:
def _file_object(bucket_prefix,product_name,year,day):
    if product_name == 'NDVI':
        file_object = bucket_prefix + 'NDVI/daily_median/' + 'med.{}.250_m_NDVI.tif'.format(day)
    elif product_name == 'pet':
        file_object = bucket_prefix + 'ETo/' + 'pet{}.tif'.format(day)
    elif product_name == 'tavg' or product_name == 'tmax' or product_name == 'tmin':
        variable_prefix = bucket_prefix + 'Temp/daily/' + product_name + '/'
        file_object = variable_prefix  + str(year) + '/' + '{0}_{1}{2}.tif'.format(product_name, year, day)
    elif product_name == 'prec':
        file_object = bucket_prefix + 'PPT/' + str(year) + '/' + '{0}_{1}{2}.tif'.format(product_name, year, day)
    else:
        file_object = bucket_prefix  + str(start_year) + '/' + f'{product_name}_' + str(start_year) + day + '.tif'
    return file_object

In [5]:
def create_s3_list_of_days_start_end(main_bucket_prefix, start_year,start_day, end_year, end_day, product_name):
    the_list = []
    years = []
    for year in (range(int(start_year),int(end_year)+1)):
        years.append(year)
    if len(years) == 1:
        for i in range(int(start_day),int(end_day)):
            day = f'{i:03d}'
            file_object = _file_object(main_bucket_prefix,product_name,start_year,day)
            the_list.append(file_object)
    elif len(years) == 2:
        for i in range(int(start_day),366):
            day = f'{i:03d}'
            file_object = _file_object(main_bucket_prefix,product_name,start_year,day)
            the_list.append(file_object)
        for i in range(1,int(end_day)):
            day = f'{i:03d}'
            file_object = _file_object(main_bucket_prefix,product_name,end_year,day)
            the_list.append(file_object)
    else:
        for i in range(int(start_day),366):
            day = f'{i:03d}'
            file_object = _file_object(main_bucket_prefix,product_name,start_year,day)
            the_list.append(file_object)
        for year in years[1:-1]:
            for i in range(1,366):
                day = f'{i:03d}'
                file_object = _file_object(main_bucket_prefix,product_name,year,day)
                the_list.append(file_object)
        for i in range(1,int(end_day)):
            day = f'{i:03d}'
            file_object = _file_object(main_bucket_prefix,product_name,end_year,day)
            the_list.append(file_object)
    return the_list

In [6]:
def xr_build_cube_concat_ds_one(tif_list, product, x, y):
    start = time()
    my_da_list =[]
    year_month_list = []
    for tif in tif_list:
        #tiffile = 's3://dev-et-data/' + tif
        tiffile = tif
        print(tiffile)
        da = xr.open_rasterio(tiffile)
        daSub = da.sel(x=x, y=y, method='nearest')
        #da = da.squeeze().drop(labels='band')
        #da.name=product
        my_da_list.append(daSub)
        tnow = time()
        elapsed = tnow - start
        print(tif, elapsed)
        year_month_list.append(_get_year_month(product, tif))

    da = xr.concat(my_da_list, dim='band')
    da = da.rename({'band':'year_month'})
    da = da.assign_coords(year_month=year_month_list)
    DS = da.to_dataset(name=product)
    return(DS)

In [7]:
main_bucket_prefix='s3://dev-et-data/in/NorthAmerica/'
start_year = '1999'
start_day = '1'
end_year = '1999'
end_day = '365'
x=-75
y=41

In [ ]:
df_list=[]
for product in ['prec','tavg', 'tmin', 'tmax', 'NDVI', 'pet']:
    print("==="*30)
    print("processing product",product)
    tif_list = create_s3_list_of_days_start_end(main_bucket_prefix, start_year,start_day, end_year, end_day, product)
    print (tif_list)
    ds_pix=xr_build_cube_concat_ds_one(tif_list, product, x, y)
    my_index = ds_pix['year_month'].values
    my_array = ds_pix[product].values
    df = pd.DataFrame(my_array, columns=[product,], index=my_index)
    df_list.append(df)

processing product prec
['s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999001.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999002.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999003.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999004.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999005.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999006.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999007.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999008.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999009.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999010.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999011.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999012.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999013.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999014.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999015.tif', 's3://dev-et-data/in/NorthAmerica/PPT/1999/pr

s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999001.tif 9.85841155052185
1999001
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999002.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999002.tif 17.880486011505127
1999002
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999003.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999003.tif 26.330998182296753
1999003
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999004.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999004.tif 34.75182509422302
1999004
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999005.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999005.tif 42.96237254142761
1999005
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999006.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999006.tif 51.18498086929321
1999006
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999007.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999007.tif 60.66846227645874
1999007
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_199

s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999058.tif 416.4090814590454
1999058
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999059.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999059.tif 426.88735795021057
1999059
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999060.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999060.tif 436.76788568496704
1999060
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999061.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999061.tif 446.519864320755
1999061
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999062.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999062.tif 455.7526795864105
1999062
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999063.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999063.tif 464.68853878974915
1999063
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999064.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999064.tif 474.59778213500977
1999064
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1

s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999115.tif 867.4296877384186
1999115
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999116.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999116.tif 876.9963583946228
1999116
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999117.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999117.tif 885.9528238773346
1999117
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999118.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999118.tif 894.0975470542908
1999118
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999119.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999119.tif 902.3776404857635
1999119
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999120.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999120.tif 910.1709566116333
1999120
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999121.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999121.tif 919.0279490947723
1999121
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999

s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999172.tif 1268.9218900203705
1999172
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999173.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999173.tif 1277.3111157417297
1999173
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999174.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999174.tif 1285.8941185474396
1999174
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999175.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999175.tif 1294.3426101207733
1999175
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999176.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999176.tif 1303.5647792816162
1999176
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999177.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999177.tif 1313.7826552391052
1999177
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999178.tif
s3://dev-et-data/in/NorthAmerica/PPT/1999/prec_1999178.tif 1322.758546590805
1999178
s3://dev-et-data/in/NorthAmerica/PPT/1999/pre

...

s3://dev-et-data/in/NorthAmerica/ETo/pet071.tif 370.76413917541504
071
s3://dev-et-data/in/NorthAmerica/ETo/pet072.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet072.tif 375.8037066459656
072
s3://dev-et-data/in/NorthAmerica/ETo/pet073.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet073.tif 380.7678942680359
073
s3://dev-et-data/in/NorthAmerica/ETo/pet074.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet074.tif 386.4538767337799
074
s3://dev-et-data/in/NorthAmerica/ETo/pet075.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet075.tif 391.9212076663971
075
s3://dev-et-data/in/NorthAmerica/ETo/pet076.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet076.tif 397.21059560775757
076
s3://dev-et-data/in/NorthAmerica/ETo/pet077.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet077.tif 402.9271605014801
077
s3://dev-et-data/in/NorthAmerica/ETo/pet078.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet078.tif 408.5650975704193
078
s3://dev-et-data/in/NorthAmerica/ETo/pet079.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet079.tif 414.16

s3://dev-et-data/in/NorthAmerica/ETo/pet141.tif 755.525502204895
141
s3://dev-et-data/in/NorthAmerica/ETo/pet142.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet142.tif 761.8493690490723
142
s3://dev-et-data/in/NorthAmerica/ETo/pet143.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet143.tif 768.3033003807068
143
s3://dev-et-data/in/NorthAmerica/ETo/pet144.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet144.tif 775.6169707775116
144
s3://dev-et-data/in/NorthAmerica/ETo/pet145.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet145.tif 782.4040558338165
145
s3://dev-et-data/in/NorthAmerica/ETo/pet146.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet146.tif 789.6968193054199
146
s3://dev-et-data/in/NorthAmerica/ETo/pet147.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet147.tif 798.2542116641998
147
s3://dev-et-data/in/NorthAmerica/ETo/pet148.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet148.tif 802.9256589412689
148
s3://dev-et-data/in/NorthAmerica/ETo/pet149.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet149.tif 807.86081

s3://dev-et-data/in/NorthAmerica/ETo/pet211.tif 1133.5582928657532
211
s3://dev-et-data/in/NorthAmerica/ETo/pet212.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet212.tif 1138.0139915943146
212
s3://dev-et-data/in/NorthAmerica/ETo/pet213.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet213.tif 1142.9577000141144
213
s3://dev-et-data/in/NorthAmerica/ETo/pet214.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet214.tif 1146.9933123588562
214
s3://dev-et-data/in/NorthAmerica/ETo/pet215.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet215.tif 1150.9175877571106
215
s3://dev-et-data/in/NorthAmerica/ETo/pet216.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet216.tif 1155.0530178546906
216
s3://dev-et-data/in/NorthAmerica/ETo/pet217.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet217.tif 1158.843430519104
217
s3://dev-et-data/in/NorthAmerica/ETo/pet218.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet218.tif 1163.3553812503815
218
s3://dev-et-data/in/NorthAmerica/ETo/pet219.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet219.tif 1

s3://dev-et-data/in/NorthAmerica/ETo/pet280.tif 1469.4998850822449
280
s3://dev-et-data/in/NorthAmerica/ETo/pet281.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet281.tif 1475.1322846412659
281
s3://dev-et-data/in/NorthAmerica/ETo/pet282.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet282.tif 1480.4829902648926
282
s3://dev-et-data/in/NorthAmerica/ETo/pet283.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet283.tif 1485.9032242298126
283
s3://dev-et-data/in/NorthAmerica/ETo/pet284.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet284.tif 1491.2026998996735
284
s3://dev-et-data/in/NorthAmerica/ETo/pet285.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet285.tif 1496.6578981876373
285
s3://dev-et-data/in/NorthAmerica/ETo/pet286.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet286.tif 1502.635886669159
286
s3://dev-et-data/in/NorthAmerica/ETo/pet287.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet287.tif 1508.840347290039
287
s3://dev-et-data/in/NorthAmerica/ETo/pet288.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet288.tif 15

s3://dev-et-data/in/NorthAmerica/ETo/pet349.tif 1819.5160057544708
349
s3://dev-et-data/in/NorthAmerica/ETo/pet350.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet350.tif 1825.9392728805542
350
s3://dev-et-data/in/NorthAmerica/ETo/pet351.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet351.tif 1831.4448091983795
351
s3://dev-et-data/in/NorthAmerica/ETo/pet352.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet352.tif 1837.6075015068054
352
s3://dev-et-data/in/NorthAmerica/ETo/pet353.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet353.tif 1842.6775505542755
353
s3://dev-et-data/in/NorthAmerica/ETo/pet354.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet354.tif 1847.5410749912262
354
s3://dev-et-data/in/NorthAmerica/ETo/pet355.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet355.tif 1852.8963470458984
355
s3://dev-et-data/in/NorthAmerica/ETo/pet356.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet356.tif 1857.9903900623322
356
s3://dev-et-data/in/NorthAmerica/ETo/pet357.tif
s3://dev-et-data/in/NorthAmerica/ETo/pet357.tif 

In [ ]:
df_reset_list = []
for dframe in df_list:
    print (dframe)
    df_reset = dframe.set_index(df_list[0].index)
    print (df_reset)
    df_reset_list.append(df_reset)
df_veget = pd.concat(df_reset_list, axis=1)

df_veget['NDVI'] *= 0.0001
df_veget

In [ ]:
for static_product in ['awc', 'por', 'fc', 'intercept', 'water']:
    if static_product == 'awc' or static_product == 'por' or static_product == 'fc':
        file_object = ['s3://dev-et-data/in/NorthAmerica/Soil/' + '{}_NA_mosaic.tif'.format(static_product)]
    elif static_product == 'intercept':
        file_object = ['s3://dev-et-data/in/NorthAmerica/Soil/' + 'intercept_filled_0_na.tif']
    else:
        file_object = ['s3://dev-et-data/in/NorthAmerica/' + 'global_water_mask_inland.tif']
    ds_pix=xr_build_cube_concat_ds_one(file_object, static_product, x, y)
    df_veget['{}'.format(static_product)] = ds_pix[static_product].values[0]
print (df_veget)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15,12))

# first graph - prec and NDVI on two separate axis
axs[0].bar(df_veget.index, df_veget["prec"], color = 'lightskyblue')
ax0 = axs[0].twinx()
ax0.plot(df_veget.index, df_veget["NDVI"], color = 'seagreen')
axs[0].set_ylabel("prec, mm")
ax0.set_ylabel("NDVI")
ax0.set_ylim([0,1])
axs[0].set_xticks([])
ax0.set_xticks([])
axs[0].set_xlim([df_veget.index[0], df_veget.index[-1]])
ax0.set_xlim([df_veget.index[0], df_veget.index[-1]])

# second graph - Tmin, Tavg, Tmax
low_threshold = np.array([0 for i in range(len(df_veget))])
axs[1].plot(df_veget.index, low_threshold,  '--', color = 'dimgray', linewidth=0.8)
high_threshold = np.array([6 for i in range(len(df_veget))])
axs[1].plot(df_veget.index, high_threshold,  '--', color = 'dimgray', linewidth=0.8)
axs[1].plot(df_veget.index, df_veget["tmin"], color = 'navy', linewidth=2.5)
axs[1].plot(df_veget.index, df_veget["tavg"], color = 'slategray', linewidth=2.5)
axs[1].plot(df_veget.index, df_veget["tmax"], color = 'red', linewidth=2.5)
axs[1].set_ylabel("T, deg C")
axs[1].set_xticks([])
axs[1].set_xlim([df_veget.index[0], df_veget.index[-1]])

# third graph - ETo
axs[2].plot(df_veget.index, df_veget["pet"], color = 'goldenrod')
axs[2].set_ylabel("ET, mm")
axs[2].set_xticks(axs[2].get_xticks()[::10])
axs[2].set_xlim([df_veget.index[0], df_veget.index[-1]])

# make a legend based on color inputs
ppt = mpatches.Patch(color='lightskyblue', label='PPT')
ndvi = mpatches.Patch(color='seagreen', label='NDVI')
tmax = mpatches.Patch(color='red', label='Tmax')
tavg = mpatches.Patch(color='slategray', label='Tavg')
tmin = mpatches.Patch(color='navy', label='Tmin')
eto = mpatches.Patch(color='goldenrod', label='ETo')
plt.legend(handles=[ppt, ndvi, tmax, tavg, tmin, eto])

fig.autofmt_xdate()
plt.tight_layout()
#fig.savefig('test.png', dpi=300, format='png', frameon=True)